<a href="https://colab.research.google.com/github/gurmaaan/MedNN/blob/master/Trained_CNN_no_changes_in_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Подключение Google диска

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/My\ Drive/MedNN

In [1]:
#pytorch
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import models

# general
import os
from collections import defaultdict, Counter
import time
from datetime import datetime
import json
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.metrics import accuracy_score
from pprint import pprint

## Загрузка данных 

In [2]:
#@title Параметры загрузчика

batch_size = 24  #@param {type: "slider", min: 4, max: 128}

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

resize_s = 256  #@param {type: "number"}
crop_s = 224  #@param {type: "number"}

train_path = "img/train" #@param ["img/train", "img/train_65", "img/train_enriched"] {allow-input: true}


In [3]:
def get_transforms(resize = 256, center_crop = 224):
    tr_dict = {
        'train': 
        [
            transforms.Resize(resize),
            transforms.CenterCrop(center_crop),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ],
        'valid': 
        [
            transforms.Resize(resize),
            transforms.CenterCrop(center_crop),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ],
        'test': 
        [
            transforms.Resize(resize),
            transforms.CenterCrop(center_crop),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]
    }
    return tr_dict

In [4]:
def update_data(path_dict, resize = 256, center_crop = 224, batch_size = 4):
    data_transforms = get_transforms(resize, center_crop)
    image_datasets = {x: ImageFolder(paths[x], transforms.Compose(data_transforms[x])) for x in paths}
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in paths}
    return data_transforms, image_datasets, dataloaders

In [5]:
paths = {
    "train" : train_path,
    "test" : "img/test",
    "valid" : "img/valid"
}

In [6]:
data_transforms, image_datasets, dataloaders = update_data(paths, resize_s, crop_s, batch_size)

In [9]:
for i, (inputs, labels) in enumerate(dataloaders["train"]):
    print(labels)
    if i > 20:
        break

tensor([4, 1, 4, 5, 6, 4, 4, 4, 4, 4, 4, 4, 5, 6, 5, 4, 4, 4, 4, 4, 3, 4, 7, 3])
tensor([5, 4, 5, 4, 5, 1, 4, 4, 4, 4, 4, 3, 4, 5, 4, 4, 6, 5, 3, 4, 4, 4, 4, 4])
tensor([6, 4, 4, 4, 4, 4, 5, 5, 4, 3, 5, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 7, 4, 4])
tensor([5, 4, 5, 1, 4, 5, 4, 4, 5, 4, 4, 4, 1, 6, 4, 4, 7, 4, 4, 3, 4, 4, 4, 4])
tensor([4, 1, 4, 4, 4, 4, 1, 2, 2, 4, 4, 7, 4, 5, 4, 6, 4, 4, 4, 4, 4, 5, 4, 4])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 6, 3, 4, 4, 4, 4, 4, 4, 4])
tensor([3, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 0, 4, 4, 5, 3, 4, 4, 1, 4, 4])
tensor([4, 4, 7, 4, 3, 4, 3, 4, 4, 4, 4, 4, 3, 4, 5, 4, 4, 4, 4, 5, 4, 5, 4, 4])
tensor([4, 1, 4, 4, 4, 4, 4, 4, 5, 4, 5, 5, 3, 1, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4])
tensor([1, 4, 4, 4, 4, 4, 4, 4, 4, 5, 3, 3, 4, 1, 4, 3, 4, 4, 4, 4, 4, 5, 4, 4])
tensor([4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 5, 3, 4, 3, 4, 4, 2, 3, 3, 4, 6, 4])
tensor([4, 4, 4, 4, 5, 0, 4, 4, 3, 3, 5, 4, 3, 4, 4, 4, 4, 4, 7, 4, 4, 4, 4, 4])
tensor([4, 4, 4, 1, 1, 7, 3,

In [10]:
class_names = image_datasets['train'].classes
class_names

['actinic keratosis',
 'basal cell carcinoma',
 'dermatofibroma',
 'melanoma',
 'nevus',
 'pigmented benign keratosis',
 'squamous cell carcinoma',
 'vascular lesion']

TODO: проверить как веса в data loader будут влиять на качество

In [11]:
# target = torch.tensor(image_datasets["train"].targets)
# class_sample_count = torch.tensor([(target == t).sum() for t in torch.unique(target, sorted=True)])
# weight = 1. / class_sample_count.float()
# samples_weight = torch.tensor([weight[t] for t in target])

# sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weight, len(samples_weight))
# dataloaders["train"] = DataLoader(image_datasets["train"], batch_size=batch_size, num_workers=4, sampler=sampler)

In [13]:
dataset_sizes = {x: len(image_datasets[x]) for x in paths}
pprint(dataset_sizes)
for p in dataset_sizes:
    if dataset_sizes[p] % batch_size == 1:
        print(f"Incorrect bath_size. {p} size % batch_size = {dataset_sizes[p]} % {batch_size} == 1")

{'test': 2504, 'train': 5633, 'valid': 1878}


In [14]:
class_counts = defaultdict(dict)
for p in paths:
    for cl in class_names:
        class_counts[p][cl] = len(os.listdir(f"{paths[p]}/{cl}"))
pd.DataFrame(class_counts)

,train,test,valid
actinic keratosis,71,30,29
basal cell carcinoma,297,127,90
dermatofibroma,65,28,22
melanoma,608,303,202
nevus,3760,1673,1272
pigmented benign keratosis,632,259,208
squamous cell carcinoma,120,47,30
vascular lesion,80,37,25


## Device

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("CPU device")

GeForce GTX 970


## Визуализация 

In [16]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    
    #revert normalization
    inp = std * inp + mean
    
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(16, 16))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)

In [ ]:
imgs, labels = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(imgs)

imshow(out, title=[class_names[x] for x in labels])

## Функция обучения


In [17]:
def train_model(model, criterion, optimizer, scheduler = None, num_epochs=25):
    start_time = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    stat_dict = {
        "train" :{ "epoch" : [], "loss" : [], "acc" : [] },
        "valid" :{ "epoch" : [], "loss" : [], "acc" : [] }
    }
    expect = 0
    print(f"Начало обучения. Проход в {num_epochs} эпох.")
    for epoch in range(num_epochs):
        estart_time = time.time()
        # У каждой эпохи есть фаза обучения и фаза валидации 
        for phase in ["train", "valid"]:
            is_train_phase = (phase == "train")
            # Ставим модель в нужный режим в зависимости от фазы 
            model.train(is_train_phase)
            running_loss = 0.0
            running_corrects = 0
            
            # Проходим по датасету партиями размером с батч
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                # Обнуляем градиенты вычесленные на предыдущем батче, иначе они будут складываться один за другим 
                optimizer.zero_grad()
                # Прямой проход Общая для обучения и валидации. 
                # Если фаза обучения, то все градиенты (частные производные функции потерь) 
                # будут вычислены чтобы вычесть их из текущих весов. Если валидация - вычисляться не будут
                with torch.set_grad_enabled(is_train_phase):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # Реализация обратного распространения ошибки 
                    if is_train_phase:
                        # Рассчет всех частных производных dloss_dw1(w1_0). Внутри батча усредняются
                        loss.backward()
                        # Обновление весов (w1_1 = w1_0 - lr * dloss_dw1(w1_0) и так далее
                        optimizer.step() 

                # Расчет точности и потери на конкретной эпохе 
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if is_train_phase and (scheduler is not None):
                scheduler.step()
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            # Сохраняем веса если точность выше чем была
            if (not is_train_phase) and (epoch_acc > best_acc):
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
            stat_dict[phase]["epoch"].append(epoch)
            stat_dict[phase]["loss"].append(epoch_loss)
            stat_dict[phase]["acc"].append(epoch_acc.item())
            
            te = time.time() - estart_time
            if epoch == 0:
                expect += te
                if not is_train_phase:
                    expect *= (num_epochs - 1) * 0.6
                    print(f"{datetime.now():%Y-%m-%d %H:%M}. Приблизительное оставшееся время {expect//60:2.0f}м {expect%60:2.0f}с")
                    print("***")
            if ( (epoch+1) % 5 == 0 ) and epoch != 0:        
                print(f"{epoch+1:2}. {phase}. loss: {epoch_loss:.4f}. acc: {epoch_acc:.4f}. time {te//60:2.0f}м {te%60:2.0f}с")

    print('-'*80)
    time_elapsed = time.time() - start_time
    print(f"Обучение закончено. Общее время: {time_elapsed//60:2.0f}м {time_elapsed%60:2.0f}с")
    print(f"valid acc: {best_acc * 100 :.2f}%")

    # Возвращается модель с лучшими весами 
    model.load_state_dict(best_model_wts)
    return model, stat_dict

## Функция тестирования

In [18]:
def test_model(model, print_log=True):
    result_dict = { "real" : [], "predicted" : [] }

    with torch.no_grad():
        for images, labels in dataloaders["test"]:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            for i in range(len(labels)):
                result_dict["real"].append(class_names[labels[i]])
                result_dict["predicted"].append(class_names[predicted[i]])

    result_df = pd.DataFrame(result_dict)

    acc_dict = {}
    total_acc = accuracy_score(result_df["real"], result_df["predicted"]) * 100
    acc_dict["total"] = total_acc

    for img_class in class_names:
        class_df = result_df[result_df["real"] == img_class]
        class_acc = accuracy_score(class_df["real"], class_df["predicted"]) * 100
        acc_dict[img_class] = class_acc
        if print_log:
            print(f"{img_class} ({len(class_df)}) : {class_acc :.2f}%")

    print(f"test acc: {total_acc :.2f}%")
    
    return acc_dict

## Вспомогательные функции 

In [19]:
def convert_score_to_df(score_dict, save_flag=True, save_path = "/nn", save_dir="", single_model=False):
    data = defaultdict(list)
    if len(score_dict.keys()) > 0:
        first_model_name = list(score_dict.keys())[0]
        for scope in score_dict[first_model_name]:
            data["scope"].append(scope)
            for model_name in score_dict:
                data[model_name].append(f"{score_dict[model_name][scope] : .2f}")
        
        df = pd.DataFrame(data)
        df = df.set_index("scope")
        
        if save_flag:
            output_dir = f"{save_path}/{save_dir}"
            if not os.path.isdir(output_dir):
                os.makedirs(output_dir)
            file_name = f"{first_model_name}_score.csv" if single_model else "models_score.csv"

            df.to_csv(f"{output_dir}/{file_name}", sep=';')
    else:
        df  = pd.DataFrame()
    return df

In [20]:
def save_learning_stat(loss_dict, save_path = "/nn", save_dir="", modelname="NN"):
    output_dir = f"{save_path}/{save_dir}"
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
    with open(f"{output_dir}/{modelname}.json", 'w') as json_file:
        json.dump(loss_dict, json_file)

In [21]:
def plot_learning_curve(data, model_name, save_flag=True, save_path = "/nn", save_dir=""):
    # data = json.load(open(path_json, 'r'))
    
    epochs = data["train"]["epoch"]
    
    fig, axs = plt.subplots(2, figsize=(16,12))
    # model_name = path.split('/')[-1].split('.')[0]
    fig.suptitle(model_name)
    
    epcnt = float(epochs[-1])
    step = 1
    while epcnt / step >= 20:
        step += 1

    for i, mode in enumerate(["loss", "acc"]):
        for phase in ["train", "valid"]:
            axs[i].set_title(mode)
            axs[i].plot(epochs, data[phase][mode], label=phase)
            axs[i].set_xlabel("Эпохи")
            axs[i].legend(loc=1)
            axs[i].grid(True)
            delta = 0.001
            axs[i].set_xlim(xmin=(0 - delta), xmax=(epcnt + delta))
            axs[i].set_xticks([xt for xt in range(0, int(epcnt+1), step)])

    if save_flag:
        output_dir = f"{save_path}/{save_dir}"
        if not os.path.isdir(output_dir):
            os.makedirs(output_dir)
        save_learning_stat(data, save_path, save_dir, model_name)
        plt.savefig(f"{output_dir}/{model_name}.png")

In [22]:
#@title Параметры обучения

save_path = "C:/Users/Dima/YandexDisk/EDUCATION/_Deeplom/nn" #@param {type: "string"}
save_dir = "layerTest_other_24ep" #@param {type: "string"}
#@markdown ---
num_epochs = 24 #@param {type:"slider", min:3, max:45, step:3}
#@markdown ---
start_lr = 0.01  #@param {type: "number"}
use_lr_scheduler = True #@param {type:"boolean"}
lr_scheduler_step = 7 #@param {type:"integer"}

## **ОБУЧЕНИЕ**

In [23]:
data_transforms, image_datasets, dataloaders = update_data(paths, resize_s, crop_s, batch_size)

In [24]:
model_dict = {
    "ResNeXt_50_32x4d": {"model" : models.resnext50_32x4d(pretrained=True), "type" : "fc", "epochs" : num_epochs, "lr_step" : lr_scheduler_step},
    "MnasNet_0-5" : {"model" : models.mnasnet0_5(pretrained=True), "type" : "cl", "epochs" : num_epochs, "lr_step" : lr_scheduler_step},
    "Wide_ResNet_50-2" : {"model" : models.wide_resnet50_2(pretrained=True), "type" : "fc", "epochs" : num_epochs, "lr_step" : lr_scheduler_step}
#   ------------
#     "AlexNet" : {"model" : models.alexnet(pretrained=True), "type" : "cl", "epochs" :int(num_epochs/2), "lr_step" : int(lr_scheduler_step / 2)},
#     "MobileNet_v2" : {"model" : models.mobilenet_v2(pretrained=True), "type" : "cl", "epochs" : int(num_epochs/2), "lr_step" : int(lr_scheduler_step / 2)},
#     "ResNet_152" : {"model" : models.resnet152(pretrained=True), "type" : "fc", "epochs" : int(num_epochs/3), "lr_step" : int(lr_scheduler_step / 2)},
#     "ResNeXt_101_32x8d" : {"model" : models.resnext101_32x8d(pretrained=True), "type" : "fc", "epochs" : int(num_epochs/3), "lr_step" : int(lr_scheduler_step / 2)},
#     "Wide_ResNet_101-2" : {"model" : models.wide_resnet101_2(pretrained=True), "type" : "fc", "epochs" : int(num_epochs/3), "lr_step" : int(lr_scheduler_step / 2)},
#   ------------
#     "VGG_19_bn" : {"model" : models.vgg19_bn(pretrained=True), "type" : "cl", "epochs" : num_epochs*2, "lr_step" : lr_scheduler_step * 2},
#     "MnasNet_1-0" : {"model" : models.mnasnet1_0(pretrained=True), "type" : "cl", "epochs" : num_epochs*2, "lr_step" : lr_scheduler_step * 2},
#     "Inception_v3" : {"model" : models.inception_v3(pretrained=True), "type" : "fc", "epochs" : num_epochs*2, "lr_step" : lr_scheduler_step * 2}
}

In [25]:
def learn_model(model, model_name, params_to_learn, ep_num, lr_step):
    mnl = int(len(model_name) / 2)
    print(' ' * (40 - mnl), model_name, ' ' * (40 - mnl))
    print('_'*80)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(params_to_learn, lr=start_lr, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=lr_step, gamma=0.1) if use_lr_scheduler else None

    model, learning_dict = train_model(model, criterion, optimizer, exp_lr_scheduler, ep_num)
    
    plot_learning_curve(learning_dict, model_name, True, save_path, save_dir)
    
    coeff_path = f"{save_path}/{save_dir}/{model_name}.pth"
    torch.save(model.state_dict(), coeff_path)
    model_dict[model_name]["model"] = model

    score = test_model(model, False)
    return convert_score_to_df({model_name : score}, True, save_path, save_dir, True)

In [ ]:
print('_'*80)
for model_name in model_dict:
    model = model_dict[model_name]["model"]
    
    if model_name == "Inception_v3":
        data_transforms, image_datasets, dataloaders = update_data(paths, resize=340, center_crop=299, batch_size = batch_size)
        model.aux_logits=False

    # Указываем парамтры модели для которых не требуется строить граф вычислений градиента. <=> "замораживаем веса"
    for param in model.parameters():
        param.requires_grad = False

    params_to_learn = model.parameters()

    # Последний слой модели меняется на собственный полносвязный
    if model_dict[model_name]["type"] == "fc":
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, len(class_names))
        params_to_learn = model.fc.parameters()
    else:
        num_ftrs = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_ftrs, len(class_names))
        params_to_learn = model.classifier[-1].parameters()
    # Запуск процесса обновления весов
    try:
        learn_model(model, model_name, params_to_learn, model_dict[model_name]["epochs"], model_dict[model_name]["lr_step"])
    except Exception as e:
        print(f"{model_name} failed. {str(e)}")
    print('_'*80)

________________________________________________________________________________
                                 ResNeXt_50_32x4d                                 
________________________________________________________________________________
Начало обучения. Проход в 24 эпох.
2020-01-27 01:21. Приблизительное оставшееся время 24м 42с
***
 5. train. loss: 0.8719. acc: 0.7447. time  0м 45с
 5. valid. loss: 1.0264. acc: 0.7428. time  1м  1с


## Объединение результатов 

In [ ]:
def collect_stat(root=None):
    if (root is not None) and os.path.isdir(root):
        data = {}
        for file_name in os.listdir(root):
            if file_name.split('.')[-1] == "csv":
                df = pd.read_csv(os.path.join(root, file_name), sep=';')
                df = df.set_index(df.columns[0])
#                 df = df.sort_values(by=[df.columns[0]])
                
                for col in df.columns:
                    data[col] = df[col]
        
        return pd.DataFrame(data)
    else:
        return pd.DataFrame()

In [ ]:
collect_stat(f"{save_path}/{save_dir}/")

## Точность ансамбля

In [ ]:
ans_models = ["ResNeXt_101_32x8d","ResNet_152", "MnasNet_1-0", "MobileNet_v2", "Wide_ResNet_101-2"]

result_dict = defaultdict(list)

with torch.no_grad():
    for images, labels in dataloaders["test"]:
        images, labels = images.to(device), labels.to(device)
        
        for l in labels:
            result_dict["real"].append(l.item())
            
        for m_name in ans_models:
            model = model_dict[m_name]["model"]
            
            if model_dict[m_name]["type"] == "fc":
                model.fc = nn.Linear(model.fc.in_features, len(class_names))
            else:
                model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, len(class_names))
            
            model.load_state_dict(torch.load(f"{save_path}/{save_dir}/{m_name}.pth"))
            
            model = model.to(device)
            
            _, predicted = torch.max(model(images).data, 1)
            for p in predicted:
                result_dict[m_name].append(p.item())

ans_df = pd.DataFrame(result_dict)

In [ ]:
def summarize(row):
    cntr = Counter([ma for ma in row])
    if len(cntr) == len(row):
        #TODO переделать в наиболее вероятный
        return row["ResNeXt_101_32x8d"]
    else:
        return cntr.most_common(1)[0][0]

In [ ]:
ans_df["predicted"] = ans_df[ans_models].apply(lambda row : summarize(row), axis=1)

In [ ]:
total_acc = accuracy_score(ans_df["real"], ans_df["predicted"]) * 100

In [ ]:
print(f"{total_acc :.2f}")

In [ ]:
for cl in ans_df["real"].unique():
    tmp = ans_df[ans_df["real"] == cl]
    acc = accuracy_score(tmp["real"], tmp["predicted"]) * 100
    print(class_names[cl], f"{acc :.2f}")

## Все графики на одном

In [ ]:
def single_plot(ax, data, m_name, mode, step):
    epochs = data["train"]["epoch"]
    epcnt = float(epochs[-1])
    for phase in ["train", "valid"]:
#         clr = "green" if phase == "valid" else "blue"
#         clr = "light" + clr if mode == "acc" else clr
        ax.set_title(f"{m_name}", fontsize=12)
        ax.plot(epochs, data[phase][mode], label=phase)
        ax.set_ylabel(f"{mode}")
        ax.legend(loc=1)
        ax.grid(True)
        delta = 0.001
        ax.set_xlim(left=(0 - delta), right=(epcnt + delta))
        ax.set_xticks([xt for xt in range(0, int(epcnt+2), step)])

In [ ]:
fig, axs = plt.subplots(4,4, figsize=(18,12))
fig.tight_layout(h_pad=3, w_pad = 4)
i = 0
for file_name in os.listdir(f"{save_path}/{save_dir}/"):
    if file_name.split('.')[-1] == "json":
        path_json = f"{save_path}/{save_dir}/{file_name}"
        data = json.load(open(path_json, 'r'))
        model_name = file_name.split('.')[0]
        pad = 0 if i < 4 else 1
        single_plot(axs[int(i / 4) + pad][i % 4], data, model_name, "loss", 2)
        single_plot(axs[int(i / 4) + pad+1][i % 4], data, model_name, "acc", 2)
        i+=1
plt.savefig("default_24ep.png", dpi=300)

## Предсказание

In [ ]:
def predict(img_path=None, model_name="inception_v3", path="/nn/default/inception_v3.pth"):
    if os.path.isfile(img_path) and (model_name in model_dict) and os.path.isfile(img_path):
        img = Image.open(img_path)
        model = model_dict[model_name]
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 8)
        model.load_state_dict(torch.load(path))
        model.eval()
        
        img_input = transforms.Compose(data_transforms["test"])(img)
        img_input = img_input.unsqueeze(0)
        with torch.no_grad():
            output = model(img_input)
            _, predicted = torch.max(output.data, 1)
            proba = torch.nn.functional.softmax(output, dim=1).tolist()[0]
            
            plt.barh(class_names, proba, align="center")
            plt.xlim(0, 1)
            plt.xticks(np.arange(0, 1, 0.1))
            plt.grid(True, axis="x")
            
            for i, p in enumerate(proba):
                print(f"{p : .10f} - {class_names[i]}")
                plt.text(p, i, f"{p : .10f}")
        return class_names[predicted[0]]
    else:
        return None